**Smith Waterman Algorithm Implementation**

In [ ]:
import itertools
import numpy as np

# a x b is the size of the scoring matrix
# match_score, mismatch_score and gap_cost are defined
# we check the local alignment of b with a
def matrix(a, b, match_score=3, mismatch_score=-3, gap_cost=2):
#     initialize the 2D array and fill with zeros
    H = np.zeros((len(a) + 1, len(b) + 1), np.int)

#     finding the candidate diagonal, left, and up values
    for i, j in itertools.product(range(1, H.shape[0]), range(1, H.shape[1])):
        diagonal = H[i - 1, j - 1] + (match_score if a[i - 1] == b[j - 1] else mismatch_score)     
        left = H[i - 1, j] - gap_cost                                                             
        up = H[i, j - 1] - gap_cost                                                               
        H[i, j] = max(diagonal, left, up, 0)
    return H

In [ ]:
def traceback(H, b, b_='', old_i=0):
    # flip H to get index of **last** occurrence of H.max() with np.argmax()
    H_flip = np.flip(np.flip(H, 0), 1)
    i_, j_ = np.unravel_index(H_flip.argmax(), H_flip.shape)
    i, j = np.subtract(H.shape, (i_ + 1, j_ + 1))  # (i, j) are the **last** indice of H.max()
    if H[i, j] == 0:               # recursion break point
        return b_,j
    b_ = b[j - 1] + '-' + b_ if old_i - i > 1 else b[j - 1] + b_   # inserting a gap if backtracking in the same direction
    return traceback(H[0:i, 0:j], b, b_, i)             
# we do this as a recursion to keep calling the traceback function until a zero is obtained to stop the backtracking


In [ ]:
def smith_waterman(a, b, match_score=3,mismatch_score=-3, gap_cost=2):
    a, b = a.upper(), b.upper()
    H = matrix(a, b, match_score,mismatch_score, gap_cost)
    b_, pos = traceback(H, b)
    return pos, pos + len(b_)

def difference(str1, str2):
    diff = []
    for a, b in zip(str1, str2):
        if a != b:
            diff.append(a+' > '+b)
    return diff

**Example showing the process in Smith Waterman Algorithm**

In [ ]:
# prints correct scoring matrix for example
print(matrix('GGTTGACT', 'TGTTACGG'))

a, b = 'ggttgact', 'tgttacgg'
H = matrix(a, b)
print(traceback(H, b)) # ('gtt-ac', 1)

a, b = 'GGTTGACT', 'TGTTACGG'
start, end = smith_waterman(a, b)
print(a[start:end])     # GTTGAC



[[ 0  0  0  0  0  0  0  0  0]
 [ 0  0  3  1  0  0  0  3  3]
 [ 0  0  3  1  0  0  0  3  6]
 [ 0  3  1  6  4  2  0  1  4]
 [ 0  3  1  4  9  7  5  3  2]
 [ 0  1  6  4  7  6  4  8  6]
 [ 0  0  4  3  5 10  8  6  5]
 [ 0  0  2  1  3  8 13 11  9]
 [ 0  3  1  5  4  6 11 10  8]]
('gtt-ac', 1)
GTTGAC


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if __name__ == '__main__':


In [ ]:
print('Mutations ')
print(difference(a,b))
print([i for i in range(len(a)) if a[i] != b[i]])

Mutations 
['G > T', 'G > A', 'A > C', 'C > G', 'T > G']
[0, 4, 5, 6, 7]


**Aligning a Patient Gene DNA Seqeunce and Finding Mutations**

In [ ]:
# import os

from google.colab import drive
drive.mount('/content/gdrive')

import pandas as pd

skin_cancer = pd.read_csv('/content/gdrive/My Drive/Research Colab/output/preprocessed_seqs.csv', delimiter = ',')
# skin_cancer = pd.read_csv('/content/gdrive/My Drive/Research Colab/output/mutated_genes.csv', delimiter = ',')

skin_cancer.head()

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


,GENE,ID_SAMPLE,ERRONEOUS_SEQ
0,BRAF,2380400,ATGGCGGCGCTGAGCGGTGGCGGTGGTGGCGGCGCGCAGCCGGGCC...
1,BRAF,2340114,ATGGCGGCGCTGAGCGGTGGCGGTGGTGGCGGCGCGGAGCCGGGCC...
2,BRAF,1154125,ATCGCGGCGCTGAGCGGTGGCGGTGGTGTCGGCGCGGAGCCGGGCC...
3,BRAF,1548870,ATGGCGGCGCTGAGCGGTGGCGGTGGTGGCGGCGCGGAGCCGGGCC...
4,BRAF,2603643,ATGGCGGCGCTGATCGGTGGCGGTGGTGGCGGCGCGGAGCCGGGCC...


In [ ]:
# randomly selecting a patient gene from the mutated gene dataset
import random

size=len(skin_cancer)
random_datapoint_index = random.randint(0, size)
# random_datapoint_index = 7287
print(random_datapoint_index)
print(skin_cancer.iloc[random_datapoint_index]['ID_SAMPLE'])

1722
2121704


In [ ]:
# obtaining the reference gene from COSMIC API
from string import Template
import requests

BASE_URL = 'https://cancer.sanger.ac.uk/cosmic/'
SEQ_URL = Template('sequence?ln=$gene&type=$_type')

def get_gene_seq(gene, _type):
  URL = BASE_URL + SEQ_URL.substitute(gene=gene,_type=_type)
  print(f'GET ---> {URL}')
  request = requests.get(url = URL)
  sequence = request.text
  sequence = sequence.split('\n')
  sequence = ''.join(sequence[1:len(sequence)-1])
  print(sequence)
  print(len(sequence))
  return sequence;

gene_name = skin_cancer.iloc[random_datapoint_index]['GENE']
# print(gene_name)
gene = get_gene_seq(gene_name,'cdna')
# gene = get_gene_seq('BRAF','cdna')

GET ---> https://cancer.sanger.ac.uk/cosmic/sequence?ln=NRAS&type=cdna
ATGACTGAGTACAAACTGGTGGTGGTTGGAGCAGGTGGTGTTGGGAAAAGCGCACTGACAATCCAGCTAATCCAGAACCACTTTGTAGATGAATATGATCCCACCATAGAGGATTCTTACAGAAAACAAGTGGTTATAGATGGTGAAACCTGTTTGTTGGACATACTGGATACAGCTGGACAAGAAGAGTACAGTGCCATGAGAGACCAATACATGAGGACAGGCGAAGGCTTCCTCTGTGTATTTGCCATCAATAATAGCAAGTCATTTGCGGATATTAACCTCTACAGGGAGCAGATTAAGCGAGTAAAAGACTCGGATGATGTACCTATGGTGCTAGTGGGAAACAAGTGTGATTTGCCAACAAGGACAGTTGATACAAAACAAGCCCACGAACTGGCCAAGAGTTACGGGATTCCATTCATTGAAACCTCAGCCAAGACCAGACAGGGTGTTGAAGATGCTTTTTACACACTGGTAAGAGAAATACGCCAGTACCGAATGAAAAAACTCAACAGCAGTGATGATGGGACTCAGGGTTGTATGGGATTGCCATGTGTGGTGATGTAA
570


In [ ]:
patient_dna = skin_cancer.iloc[random_datapoint_index]['ERRONEOUS_SEQ']
# patient_dna = skin_cancer.iloc[random_datapoint_index]['MUTATED_SEQ']
# print(len(patient_dna))
# patient_dna = 'ATGGCGGCGCTGAGCGGTGGCGGTGGTGGCGGCGCGGAGCCGGGCCAGGCTCTGTTCAACGGGGACATGGAGCCCGAGGCCGGCGCCGGCGCCGGCGCCGCGGCCTCTTCGGCTGCGGACCCTGCCATTCCGGAGGAGGTGTGGAATATCAAACAAATGATTAAGTTGACACAGGAACATATAGAGGCCCTATTGGACAAATTTGGTGGGGAGCATAATCCACCATCAATATATCTGGAGGCCTATGAAGAATACACCAGCAAGCTAGATGCACTCCAACAAAGAGAACAACAGTTATTGGAATCTCTGGGGAACGGAACTGATTTTTCTGTTTCTAGCTCTGCATCAATGGATACCGTTACATCTTCTTCCTCTTCTAGCCTTTCAGTGCTACCTTCATCTCTTTCAGTTTTTCAAAATCCCACAGATGTGGCACGGAGCAACCCCAAGTCACCACAAAAACCTATCGTTAGAGTCTTCCTGCCCAACAAACAGAGGACAGTGGTACCTGCAAGGTGTGGAGTTACAGTCCGAGACAGTCTAAAGAAAGCACTGATGATGAGAGGTCTAATCCCAGAGTGCTGTGCTGTTTACAGAATTCAGGATGGAGAGAAGAAACCAATTGGTTGGGACACTGATATTTCCTGGCTTACTGGAGAAGAATTGCATGTGGAAGTGTTGGAGAATGTTCCACTTACAACACACAACTTTGTACGAAAAACGTTTTTCACCTTAGCATTTTGTGACTTTTGTCGAAAGCTGCTTTTCCAGGGTTTCCGCTGTCAAACATGTGGTTATAAATTTCACCAGCGTTGTAGTACAGAAGTTCCACTGATGTGTGTTAATTATGACCAACTTGATTTGCTGTTTGTCTCCAAGTTCTTTGAACACCACCCAATACCACAGGAAGAGGCGTCCTTAGCAGAGACTGCCCTAACATCTGGATCATCCCCTTCCGCACCCGCCTCGGACTCTATTGGGCCCCAAATTCTCACCAGTCCGTCTCCTTCAAAATCCATTCCAATTCCACAGCCCTTCCGACCAGCAGATGAAGATCATCGAAATCAATTTGGGCAACGAGACCGATCCTCATCAGCTCCCAATGTGCATATAAACACAATAGAACCTGTCAATATTGATGACTTGATTAGAGACCAAGGATTTCGTGGTGATGGAGGATCAACCACAGGTTTGTCTGCTACCCCCCCTGCCTCATTACCTGGCTCACTAACTAACGTGAAAGCCTTACAGAAATCTCCAGGACCTCAGCGAGAAAGGAAGTCATCTTCATCCTCAGAAGACAGGAATCGAATGAAAACACTTGGTAGACGGGACTCGAGTGATGATTGGGAGATTCCTGATGGGCAGATTACAGTGGGACAAAGAATTGGATCTGGATCATTTGGAACAGTCTACAAGGGAAAGTGGCATGGTGATGTGGCAGTGAAAATGTTGAATGTGACAGCACCTACACCTCAGCAGTTACAAGCCTTCAAAAATGAAGTAGGAGTACTCAGGAAAACACGACATGTGAATATCCTACTCTTCATGGGCTATTCCACAAAGCCACAACTGGCTATTGTTACCCAGTGGTGTGAGGGCTCCAGCTTGTATCACCATCTCCATATCATTGAGACCAAATTTGAGATGATCAAACTTATAGATATTGCACGACAGACTGCACAGGGCATGGATTACTTACACGCCAAGTCAATCATCCACAGAGACCTCAAGAGTAATAATATATTTCTTCATGAAGACCTCACAGTAAAAATAGGTGATTTTGGTCTAGCTACAGAGAAATCTCGATGGAGTGGGTCCCATCAGTTTGAACAGTTGTCTGGATCCATTTTGTGGATGGCACCAGAAGTCATCAGAATGCAAGATAAAAATCCATACAGCTTTCAGTCAGATGTATATGCATTTGGAATTGTTCTGTATGAATTGATGACTGGACAGTTACCTTATTCAAACATCAACAACAGGGACCAGATAATTTTTATGGTGGGACGAGGATACCTGTCTCCAGATCTCAGTAAGGTACGGAGTAACTGTCCAAAAGCCATGAAGAGATTAATGGCAGAGTGCCTCAAAAAGAAAAGAGATGAGAGACCACTCTTTCCCCAAATTCTCGCCTCTATTGAGCTGCTGGCCCGCTCATTGCCAAAAATTCACCGCAGTGCATCAGAACCCTCCTTGAATCGGGCTGGTTTCCAAACAGAGGATTTTAGTCTATATGCTTGTGCTTCTCCAAAAACACCCATCCAGGCAGGGGGATATGGTGCGTTTCCTGTCCACTGA'

In [ ]:
# H1 = matrix(gene, patient_dna,1,-2,-2)
# print(H1)

# print(traceback(H1, patient_dna))
# start, end = smith_waterman(gene, patient_dna,1,-2,-2)
# print(string_a[gene:end])

# patient_dna = 'AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA'
# gene = 'AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA'
print(len(gene))
print('\nMutations ')
print(difference(gene,patient_dna))
# n=0
print([i+1 for i in range(len(gene)) if gene[i] != patient_dna[i]])

570

Mutations 
['T > A', 'C > A', 'G > C']
[71, 181, 518]


In [ ]:
print(len([i+1 for i in range(len(gene)) if gene[i] != patient_dna[i]]))

3


**Evaluating whether the obtained mutation is correct**

In [ ]:
# raw_data = pd.read_csv('Data/skin V94_38_MUTANTCENSUS.csv', delimiter = ',',encoding='cp1252')

# import skin cancer patients dataset
raw_data = pd.read_csv('/content/gdrive/My Drive/Research Colab/data/skin V94_38_MUTANTCENSUS.csv', delimiter = ',',encoding='cp1252')

In [ ]:
# sample_id=skin_cancer.iloc[random_datapoint_index]['ID_SAMPLE']
sample_id=2121747
sample_mutations = raw_data[raw_data[' ID_SAMPLE'].isin([sample_id])]
# sample_mutations = sample_mutations[sample_mutations['GENE_NAME'].isin([gene_name])]
sample_mutations = sample_mutations[sample_mutations['GENE_NAME'].isin(['BRAF'])]
# print(sample_mutations)
if len(sample_mutations) != 1:
    print('No of mutations for sample '+str(sample_id)+ ': '+str(len(sample_mutations))) 
    for record in sample_mutations.iterrows():
        print(record[0][' MUTATION_DESCRIPTION'])
else:
    print(sample_mutations.iloc[0][' MUTATION_CDS'])

c.1799T>A
